In [2]:
import requests
import urllib3
import time
from datetime import datetime, timedelta

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ===== API Configuration =====
TOKEN_URL = 'https://qa.onprem.icedq.com/auth/realms/icedq.qa/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu.g',
    'password': 'Admin@1234'
}

copyRule_url = 'https://qa.onprem.icedq.com/api/v1/rules:batchCopy'
common_headers = {
    'accept': 'application/json, text/plain, */*',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-36655681-7ca9-58e1-b119-71ddb41cad1e'
}

copyRule_Payload = {
    "destinationId": "fldr-02bbee98-ce60-5fad-858d-1f98786ab45b",
    "items": [
        {"resource": "Rule", "id": "rule-a9f0f1f2-21ac-566e-87cc-9db612c72fdb"},
        {"resource": "Rule", "id": "rule-96e5a527-c5ce-55c2-9964-edd97dc07ed9"},
        {"resource": "Rule", "id": "rule-ce4f1d4a-6996-5cd3-95bc-4b967b95a000"},
        {"resource": "Rule", "id": "rule-c7f469dc-72a6-5b88-9323-535ba12f497a"}
    ]
}

# ===== Token Management =====
def get_access_token():
    """Fetch a new token and record its expiry time."""
    resp = requests.post(TOKEN_URL, data=token_data, verify=False)
    if resp.status_code != 200:
        raise Exception(f"❌ Failed to get token: {resp.text}")

    data = resp.json()
    token = data.get("access_token")
    expires_in = data.get("expires_in", 300)  # default 5 mins if not provided
    if not token:
        raise Exception("❌ Token not found in response.")

    expiry_time = datetime.now() + timedelta(seconds=expires_in - 30)  # refresh 30 sec before expiry
    return token, expiry_time

# ===== Execution Loop =====
interval = 10  # seconds between calls
count = 12     # total calls
task_ids = []

# Initial token
access_token, token_expiry = get_access_token()
headers = {**common_headers, 'authorization': f"Bearer {access_token}"}

for i in range(count):
    # Refresh token if expired
    if datetime.now() >= token_expiry:
        print("🔄 Token expired. Fetching new one...")
        access_token, token_expiry = get_access_token()
        headers['authorization'] = f"Bearer {access_token}"

    try:
        response = requests.post(copyRule_url, headers=headers, json=copyRule_Payload, verify=False)

        if response.status_code == 202:
            task_id = response.json().get('taskInstanceId')
            if task_id:
                task_ids.append(task_id)
                print(f"[{i+1}] ✅ Success: {task_id}")
            else:
                print(f"[{i+1}] ⚠ No taskInstanceId in response: {response.json()}")
        else:
            print(f"[{i+1}] ❌ Failed {response.status_code}: {response.text}")

    except requests.exceptions.RequestException as e:
        print(f"[{i+1}] 🔥 Error: {e}")

    if i < count - 1:
        time.sleep(interval)

# ===== Summary =====
print("\n📋 All collected taskInstanceIds:")
for idx, tid in enumerate(task_ids, 1):
    print(f"{idx}: {tid}")


[1] ✅ Success: tins-fd76dbee-1993-5009-8147-38cb8d2048fa
[2] ✅ Success: tins-232b8bec-8aa8-560d-a005-6bf0acfb894c
[3] ✅ Success: tins-168108d7-b9d4-50af-871e-7478f5fa3885
[4] ✅ Success: tins-40224e26-8250-57f2-bc8e-2bd13f07522e
[5] ✅ Success: tins-6197dfc1-8afd-5f54-9151-acd7932f6520
[6] ✅ Success: tins-7fce597b-4588-54f2-920a-efd1bdb4484f
[7] ✅ Success: tins-9eccdabf-802d-5ea2-9eb4-8dc42a114256
[8] ✅ Success: tins-0a2c7a53-0abc-5183-8755-657698b31a61
[9] ✅ Success: tins-15b44474-02f6-5994-8a8f-05131c5a3f8c
[10] ✅ Success: tins-8b7eadf4-8961-5daa-9a71-1b81bc6bc595
[11] ✅ Success: tins-214a2493-24cb-50dc-966b-d44496ac863d
[12] ✅ Success: tins-75a9289a-ed13-57a0-8b6a-52ee8102e63c

📋 All collected taskInstanceIds:
1: tins-fd76dbee-1993-5009-8147-38cb8d2048fa
2: tins-232b8bec-8aa8-560d-a005-6bf0acfb894c
3: tins-168108d7-b9d4-50af-871e-7478f5fa3885
4: tins-40224e26-8250-57f2-bc8e-2bd13f07522e
5: tins-6197dfc1-8afd-5f54-9151-acd7932f6520
6: tins-7fce597b-4588-54f2-920a-efd1bdb4484f
7: tins-9

In [3]:
import os
import sys
import time
import requests
import urllib3
import csv
from datetime import datetime

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

TOKEN_URL = 'https://qa.onprem.icedq.com/auth/realms/icedq.qa/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu.g',
    'password': 'Admin@1234'
}
token_response = requests.post(TOKEN_URL, data=token_data, verify=False)
access_token = token_response.json().get("access_token")
if not access_token:
    print("❌ Failed to get token.")
    exit()
GET_RULES = 'https://qa.onprem.icedq.com/api/v1/rules/search?pageNo=1&pageSize=500&sort=updatedTimestamp:desc'
headers = {
    'accept': 'application/json, text/plain, */*',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-36655681-7ca9-58e1-b119-71ddb41cad1e',
    'authorization': f"Bearer {access_token}"
}
payload = {
    "filter": [
        {
            "attribute": "folderId",
            "operator": "In",
            "datatype": "string",
            "value": "fldr-02bbee98-ce60-5fad-858d-1f98786ab45b"
        }
    ]
}
response = requests.post(GET_RULES, headers=headers, json=payload, verify=False)
rules = response.json().get("items", [])
rule_ids = [item["id"] for item in response.json().get("items", [])]
print(f"🔢 Total Rules find: {len(rule_ids)}\n")
CSV_FILE = r"C:\Users\TapshyuGanvir\OneDrive - Torana Inc\Documents\Scripts\data\rules.csv"
# Delete existing file if present
if os.path.exists(CSV_FILE):
    os.remove(CSV_FILE)
    print(f"🗑️ Deleted old file: {CSV_FILE}")

# Now create new CSV
with open(CSV_FILE, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["id", "name"])  # header
    for item in response.json().get("items", []):  # your existing rules
        writer.writerow([item["id"], item["name"]])

print(f"✅ Saved rules to {CSV_FILE}")


CREATE_WORKFLOW_URL = "https://qa.onprem.icedq.com/api/v1/workflows"
WORKFLOW_HEADERS = {
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json",
    "org-id": "org-icedq",
    "workspace-id": "wksc-36655681-7ca9-58e1-b119-71ddb41cad1e",
    "Authorization": f"Bearer {access_token}",
    "folder-id": "fldr-02bbee98-ce60-5fad-858d-1f98786ab45b"
}
BATCH_SIZE = 5
# --------------

# Step 1: Read all rules from CSV
rules = []
with open(CSV_FILE, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        rules.append({"id": row["id"], "name": row["name"]})

print(f"📥 Loaded {len(rules)} rules from CSV")

# Step 2: Split rules into batches of 5
def chunk_rules(rule_list, size):
    for i in range(0, len(rule_list), size):
        yield rule_list[i:i + size]

# Step 3: Create workflows batch by batch
for batch_num, batch in enumerate(chunk_rules(rules, BATCH_SIZE), start=1):
    activities = []
    for idx, rule in enumerate(batch, start=1):
        activities.append({
            "index": idx,
            "type": "ExecuteRule",
            "objectId": rule["id"],
            "name": rule["name"],
            "dependency": {
                "onSuccess": "continue",
                "onFailure": "continue",
                "onError": "continue"
            },
            "properties": {
                "useExitCode": True,
                "isActive": True,
                "engineType": "HighThroughput"
            },
            "overrides": [],
            "id": None
        })

    # Build description as rule names joined with " - "
    description_text = " - ".join([rule["name"] for rule in batch])

    workflow_payload = {
        "template": "Sequential",
        "code": None,
        "description": description_text,
        "properties": {"engineType": "Sequential"},
        "activities": activities,
        "workflowType": "Custom",
        "name": f"Workflow_{int(time.time())}_{batch_num}",
        "isActive": True
    }

    # Step 4: Call Create Workflow API
    response = requests.post(CREATE_WORKFLOW_URL, headers=WORKFLOW_HEADERS, json=workflow_payload, verify=False)

    print(f"✅ Created Workflow {batch_num} [{response.status_code}] with {len(batch)} activities")
    print(f"📝 Description: {description_text}")
    print(response.text)
    time.sleep(5)  # avoid same timestamp in name"


🔢 Total Rules find: 92

🗑️ Deleted old file: C:\Users\TapshyuGanvir\OneDrive - Torana Inc\Documents\Scripts\data\rules.csv
✅ Saved rules to C:\Users\TapshyuGanvir\OneDrive - Torana Inc\Documents\Scripts\data\rules.csv
📥 Loaded 92 rules from CSV
✅ Created Workflow 1 [200] with 5 activities
📝 Description: Validation rule with json_1759129165241 - Check with Parquet and XML Files_1759129164730 - Recon with Flat File Native and Flat File SQL_1759129164437 - Pushdown with Excel_1759129164232 - Validation rule with json_1759129150819
{"template":"Sequential","description":"Validation rule with json_1759129165241 - Check with Parquet and XML Files_1759129164730 - Recon with Flat File Native and Flat File SQL_1759129164437 - Pushdown with Excel_1759129164232 - Validation rule with json_1759129150819","activities":[{"index":1,"id":"act-ed0cc2fd-b913-5a33-b2bc-b78633567728","name":"Validation rule with json_1759129165241","type":"ExecuteRule","objectId":"rule-a9c54217-96dc-5b6e-8994-c1b33df16bec

In [1]:
import requests
import urllib3
import time
from datetime import datetime, timedelta

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ===== API Configuration =====
TOKEN_URL = 'https://192.168.100.81:32222/auth/realms/icedq/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}

copyRule_url = 'https://192.168.100.81:32222/api/v1/rules:batchCopy'
common_headers = {
    'accept': 'application/json, text/plain, */*',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-946e5c43-2b8f-54a8-b213-1e899d435aca'
}

copyRule_Payload = {
    "destinationId": "fldr-81439da7-6808-5608-8d1f-cca42919a134",
    "items": [
        {"resource": "Rule", "id": "rule-e88818c4-23fe-569d-9598-47cb80b90d29"},
        {"resource": "Rule", "id": "rule-2c95012d-e9a5-526d-924f-f047c623f87d"},
        {"resource": "Rule", "id": "rule-b51b5538-006b-5924-8a0f-ad31f1a752c6"},
        {"resource": "Rule", "id": "rule-c748987a-f7d2-5fb1-b72f-79b5a3e0bd48"}
    ]
}

# ===== Token Management =====
def get_access_token():
    """Fetch a new token and record its expiry time."""
    resp = requests.post(TOKEN_URL, data=token_data, verify=False)
    if resp.status_code != 200:
        raise Exception(f"❌ Failed to get token: {resp.text}")

    data = resp.json()
    token = data.get("access_token")
    expires_in = data.get("expires_in", 300)  # default 5 mins if not provided
    if not token:
        raise Exception("❌ Token not found in response.")

    expiry_time = datetime.now() + timedelta(seconds=expires_in - 30)  # refresh 30 sec before expiry
    return token, expiry_time

# ===== Execution Loop =====
interval = 10  # seconds between calls
count = 50     # total calls
task_ids = []

# Initial token
access_token, token_expiry = get_access_token()
headers = {**common_headers, 'authorization': f"Bearer {access_token}"}

for i in range(count):
    # Refresh token if expired
    if datetime.now() >= token_expiry:
        print("🔄 Token expired. Fetching new one...")
        access_token, token_expiry = get_access_token()
        headers['authorization'] = f"Bearer {access_token}"

    try:
        response = requests.post(copyRule_url, headers=headers, json=copyRule_Payload, verify=False)

        if response.status_code == 202:
            task_id = response.json().get('taskInstanceId')
            if task_id:
                task_ids.append(task_id)
                print(f"[{i+1}] ✅ Success: {task_id}")
            else:
                print(f"[{i+1}] ⚠ No taskInstanceId in response: {response.json()}")
        else:
            print(f"[{i+1}] ❌ Failed {response.status_code}: {response.text}")

    except requests.exceptions.RequestException as e:
        print(f"[{i+1}] 🔥 Error: {e}")

    if i < count - 1:
        time.sleep(interval)

# ===== Summary =====
print("\n📋 All collected taskInstanceIds:")
for idx, tid in enumerate(task_ids, 1):
    print(f"{idx}: {tid}")


[1] ✅ Success: tins-e891eac7-c5ea-5f98-9fcf-d01b3ed6e325
[2] ✅ Success: tins-76fd064c-8997-5212-8a11-737e47900bc5
[3] ✅ Success: tins-a5ba5ab8-d031-5e6a-886a-c5538ad411c6
[4] ✅ Success: tins-9cf2c893-de99-5964-9485-5cec75eb0854
[5] ✅ Success: tins-ce6d20ba-6d45-5b78-83fa-5fe4fa5927a5
[6] ✅ Success: tins-95f2dffc-e91d-5fa2-ab6a-c74f72043653
[7] ✅ Success: tins-bcc6ee46-7076-5daa-a80b-b41db2474430
[8] ✅ Success: tins-6a4e9d20-7645-5d21-8874-e84e1c7e93b5
[9] ✅ Success: tins-967c350c-c906-5775-9040-e636d9e3978d
[10] ✅ Success: tins-ff641931-4a75-5108-b647-5aace14e6196
[11] ✅ Success: tins-9d4d76d6-1b0e-5f46-a26f-dd34d33114e2
[12] ✅ Success: tins-93fc940d-db92-5a95-8c19-b621a90edaca
[13] ✅ Success: tins-0f622272-b97a-5e47-ba14-af1d2c9f063d
[14] ✅ Success: tins-e6090a2f-f85c-5a91-80eb-5548c9371168
[15] ✅ Success: tins-1a5ce852-f2bb-5f80-aef6-954fcebdef37
[16] ✅ Success: tins-7ecff808-4f35-5b7d-b265-0949eafb079d
[17] ✅ Success: tins-92c20de9-0ae0-579c-b111-b0676a0760ff
[18] ✅ Success: tins-f4

In [2]:
import os
import sys
import time
import requests
import urllib3
import csv
from datetime import datetime

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

TOKEN_URL = 'https://192.168.100.81:32222/auth/realms/icedq/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}
token_response = requests.post(TOKEN_URL, data=token_data, verify=False)
access_token = token_response.json().get("access_token")
if not access_token:
    print("❌ Failed to get token.")
    exit()
GET_RULES = 'https://192.168.100.81:32222/api/v1/rules/search?pageNo=1&pageSize=500&sort=updatedTimestamp:desc'
headers = {
    'accept': 'application/json, text/plain, */*',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-946e5c43-2b8f-54a8-b213-1e899d435aca',
    'authorization': f"Bearer {access_token}"
}
payload = {
    "filter": [
        {
            "attribute": "folderId",
            "operator": "In",
            "datatype": "string",
            "value": "fldr-81439da7-6808-5608-8d1f-cca42919a134"
        }
    ]
}
response = requests.post(GET_RULES, headers=headers, json=payload, verify=False)
rules = response.json().get("items", [])
rule_ids = [item["id"] for item in response.json().get("items", [])]
print(f"🔢 Total Rules find: {len(rule_ids)}\n")
CSV_FILE = r"C:\Users\TapshyuGanvir\OneDrive - Torana Inc\Documents\Scripts\data\81_instance\rules.csv"
# Delete existing file if present
if os.path.exists(CSV_FILE):
    os.remove(CSV_FILE)
    print(f"🗑️ Deleted old file: {CSV_FILE}")

# Now create new CSV
with open(CSV_FILE, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["id", "name"])  # header
    for item in response.json().get("items", []):  # your existing rules
        writer.writerow([item["id"], item["name"]])

print(f"✅ Saved rules to {CSV_FILE}")


CREATE_WORKFLOW_URL = "https://192.168.100.81:32222/api/v1/workflows"
WORKFLOW_HEADERS = {
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json",
    "org-id": "org-icedq",
    "workspace-id": "wksc-946e5c43-2b8f-54a8-b213-1e899d435aca",
    "Authorization": f"Bearer {access_token}",
    "folder-id": "fldr-81439da7-6808-5608-8d1f-cca42919a134"
}
BATCH_SIZE = 5
# --------------

# Step 1: Read all rules from CSV
rules = []
with open(CSV_FILE, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        rules.append({"id": row["id"], "name": row["name"]})

print(f"📥 Loaded {len(rules)} rules from CSV")

# Step 2: Split rules into batches of 5
def chunk_rules(rule_list, size):
    for i in range(0, len(rule_list), size):
        yield rule_list[i:i + size]

# Step 3: Create workflows batch by batch
for batch_num, batch in enumerate(chunk_rules(rules, BATCH_SIZE), start=1):
    activities = []
    for idx, rule in enumerate(batch, start=1):
        activities.append({
            "index": idx,
            "type": "ExecuteRule",
            "objectId": rule["id"],
            "name": rule["name"],
            "dependency": {
                "onSuccess": "continue",
                "onFailure": "continue",
                "onError": "continue"
            },
            "properties": {
                "useExitCode": True,
                "isActive": True,
                "engineType": "HighThroughput"
            },
            "overrides": [],
            "id": None
        })

    # Build description as rule names joined with " - "
    description_text = " - ".join([rule["name"] for rule in batch])

    workflow_payload = {
        "template": "Sequential",
        "code": None,
        "description": description_text,
        "properties": {"engineType": "Sequential"},
        "activities": activities,
        "workflowType": "Custom",
        "name": f"Workflow_{int(time.time())}_{batch_num}",
        "isActive": True
    }

    # Step 4: Call Create Workflow API
    response = requests.post(CREATE_WORKFLOW_URL, headers=WORKFLOW_HEADERS, json=workflow_payload, verify=False)

    print(f"✅ Created Workflow {batch_num} [{response.status_code}] with {len(batch)} activities")
    print(f"📝 Description: {description_text}")
    print(response.text)
    time.sleep(5)  # avoid same timestamp in name"


🔢 Total Rules find: 204

🗑️ Deleted old file: C:\Users\TapshyuGanvir\OneDrive - Torana Inc\Documents\Scripts\data\81_instance\rules.csv
✅ Saved rules to C:\Users\TapshyuGanvir\OneDrive - Torana Inc\Documents\Scripts\data\81_instance\rules.csv
📥 Loaded 204 rules from CSV
✅ Created Workflow 1 [200] with 5 activities
📝 Description: VAL_FILE_1760094475591 - CHECK_FILE_1760094475440 - Recon_file_1760094475305 - PUSH_File_1760094475266 - VAL_FILE_1760094463637
{"template":"Sequential","description":"VAL_FILE_1760094475591 - CHECK_FILE_1760094475440 - Recon_file_1760094475305 - PUSH_File_1760094475266 - VAL_FILE_1760094463637","activities":[{"index":1,"id":"act-0256de0c-cd44-51f4-911e-480858d70c29","name":"VAL_FILE_1760094475591","type":"ExecuteRule","objectId":"rule-c918062d-75cd-5aec-a324-42a5dceb830e","dependency":{"onSuccess":"continue","onFailure":"continue","onError":"continue"},"properties":{"useExitCode":true,"isActive":true}},{"index":2,"id":"act-fa84a6c7-7acd-59cc-abdc-e87e40845dc7"

In [5]:
import requests
import urllib3
import time
from datetime import datetime, timedelta

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ===== API Configuration =====
TOKEN_URL = 'https://192.168.100.76:32222/auth/realms/icedq/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}

copyRule_url = 'https://192.168.100.76:32222/api/v1/rules:batchCopy'
common_headers = {
    'accept': 'application/json, text/plain, */*',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-6c9d2689-9b9b-505c-9f30-6c3c68ee5818'
}

copyRule_Payload = {
    "destinationId": "fldr-751a2ab3-4d59-597c-b29a-683740635889",
    "items": [
        {"resource": "Rule", "id": "rule-58180139-9d53-5369-a042-13ddcc9cffe0"}
        
    ]
}

# ===== Token Management =====
def get_access_token():
    """Fetch a new token and record its expiry time."""
    resp = requests.post(TOKEN_URL, data=token_data, verify=False)
    if resp.status_code != 200:
        raise Exception(f"❌ Failed to get token: {resp.text}")

    data = resp.json()
    token = data.get("access_token")
    expires_in = data.get("expires_in", 300)  # default 5 mins if not provided
    if not token:
        raise Exception("❌ Token not found in response.")

    expiry_time = datetime.now() + timedelta(seconds=expires_in - 30)  # refresh 30 sec before expiry
    return token, expiry_time

# ===== Execution Loop =====
interval = 3  # seconds between calls
count = 30     # total calls
task_ids = []

# Initial token
access_token, token_expiry = get_access_token()
headers = {**common_headers, 'authorization': f"Bearer {access_token}"}

for i in range(count):
    # Refresh token if expired
    if datetime.now() >= token_expiry:
        print("🔄 Token expired. Fetching new one...")
        access_token, token_expiry = get_access_token()
        headers['authorization'] = f"Bearer {access_token}"

    try:
        response = requests.post(copyRule_url, headers=headers, json=copyRule_Payload, verify=False)

        if response.status_code == 202:
            task_id = response.json().get('taskInstanceId')
            if task_id:
                task_ids.append(task_id)
                print(f"[{i+1}] ✅ Success: {task_id}")
            else:
                print(f"[{i+1}] ⚠ No taskInstanceId in response: {response.json()}")
        else:
            print(f"[{i+1}] ❌ Failed {response.status_code}: {response.text}")

    except requests.exceptions.RequestException as e:
        print(f"[{i+1}] 🔥 Error: {e}")

    if i < count - 1:
        time.sleep(interval)

# ===== Summary =====
print("\n📋 All collected taskInstanceIds:")
for idx, tid in enumerate(task_ids, 1):
    print(f"{idx}: {tid}")


[1] ✅ Success: tins-2ea0aebd-db7d-5e6c-b0ea-dea3e67c33e3
[2] ✅ Success: tins-362605e1-fb5a-5cc7-8d5f-b38f383e747c
[3] ✅ Success: tins-ea831303-c8ca-5a58-8745-38aaaaef6322
[4] ✅ Success: tins-6f1dc3d0-6ab0-5ccc-bd0a-2438e68e400e
[5] ✅ Success: tins-00484db8-79ea-5c22-aba2-5c77431c55dc
[6] ✅ Success: tins-7f57d261-764d-5da7-b98f-a4e606068ef6
[7] ✅ Success: tins-4cbca21c-9fe8-5a51-89d6-20b99249a068
[8] ✅ Success: tins-d83db73c-5b6b-5649-9d44-131da53d6b12
[9] ✅ Success: tins-940e7bd7-1c02-59cd-b48a-436643a0f793
[10] ✅ Success: tins-57b77a90-dc64-5c9b-9f67-ad0c5c529f83
[11] ✅ Success: tins-3b496eb5-efb0-5371-9320-91515d672d5a
[12] ✅ Success: tins-d4d88f99-1173-562d-bb55-646a0844d4cb
[13] ✅ Success: tins-a290a1a8-ab37-50a1-ac20-d3f908377209
[14] ✅ Success: tins-bf832005-a6f2-552e-9a2a-dd15928b80ae
[15] ✅ Success: tins-35d08579-f543-549a-963d-4bf7313dd37d
[16] ✅ Success: tins-5ea3dc31-b505-5a4f-9451-4d9788a0e47a
[17] ✅ Success: tins-6311477c-6274-5852-8cd6-d4d6730116f0
[18] ✅ Success: tins-ff

In [6]:
import os
import sys
import time
import requests
import urllib3
import csv
from datetime import datetime

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

TOKEN_URL = 'https://192.168.100.76:32222/auth/realms/icedq/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}
token_response = requests.post(TOKEN_URL, data=token_data, verify=False)
access_token = token_response.json().get("access_token")
if not access_token:
    print("❌ Failed to get token.")
    exit()
GET_RULES = 'https://192.168.100.76:32222/api/v1/rules/search?pageNo=1&pageSize=500&sort=updatedTimestamp:desc'
headers = {
    'accept': 'application/json, text/plain, */*',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-6c9d2689-9b9b-505c-9f30-6c3c68ee5818',
    'authorization': f"Bearer {access_token}"
}
payload = {
    "filter": [
        {
            "attribute": "folderId",
            "operator": "In",
            "datatype": "string",
            "value": "fldr-751a2ab3-4d59-597c-b29a-683740635889,fldr-87110ae0-b1a4-5d0f-b369-65fe3f83b646"
        }
    ]
}
response = requests.post(GET_RULES, headers=headers, json=payload, verify=False)
rules = response.json().get("items", [])
rule_ids = [item["id"] for item in response.json().get("items", [])]
print(f"🔢 Total Rules find: {len(rule_ids)}\n")
CSV_FILE = r"C:\Users\TapshyuGanvir\OneDrive - Torana Inc\Documents\Scripts\data\81_instance\rules.csv"
# Delete existing file if present
if os.path.exists(CSV_FILE):
    os.remove(CSV_FILE)
    print(f"🗑️ Deleted old file: {CSV_FILE}")

# Now create new CSV
with open(CSV_FILE, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["id", "name"])  # header
    for item in response.json().get("items", []):  # your existing rules
        writer.writerow([item["id"], item["name"]])

print(f"✅ Saved rules to {CSV_FILE}")


CREATE_WORKFLOW_URL = "https://192.168.100.76:32222/api/v1/workflows"
WORKFLOW_HEADERS = {
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json",
    "org-id": "org-icedq",
    "workspace-id": "wksc-6c9d2689-9b9b-505c-9f30-6c3c68ee5818",
    "Authorization": f"Bearer {access_token}",
    "folder-id": "fldr-751a2ab3-4d59-597c-b29a-683740635889"
}
BATCH_SIZE = 5
# --------------

# Step 1: Read all rules from CSV
rules = []
with open(CSV_FILE, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        rules.append({"id": row["id"], "name": row["name"]})

print(f"📥 Loaded {len(rules)} rules from CSV")

# Step 2: Split rules into batches of 5
def chunk_rules(rule_list, size):
    for i in range(0, len(rule_list), size):
        yield rule_list[i:i + size]

# Step 3: Create workflows batch by batch
for batch_num, batch in enumerate(chunk_rules(rules, BATCH_SIZE), start=1):
    activities = []
    for idx, rule in enumerate(batch, start=1):
        activities.append({
            "index": idx,
            "type": "ExecuteRule",
            "objectId": rule["id"],
            "name": rule["name"],
            "dependency": {
                "onSuccess": "continue",
                "onFailure": "continue",
                "onError": "continue"
            },
            "properties": {
                "useExitCode": True,
                "isActive": True,
                "engineType": "HighThroughput"
            },
            "overrides": [],
            "id": None
        })

    # Build description as rule names joined with " - "
    description_text = " - ".join([rule["name"] for rule in batch])

    workflow_payload = {
        "template": "Sequential",
        "code": None,
        "description": description_text,
        "properties": {"engineType": "Sequential"},
        "activities": activities,
        "workflowType": "Custom",
        "name": f"Workflow_{int(time.time())}_{batch_num}",
        "isActive": True
    }

    # Step 4: Call Create Workflow API
    response = requests.post(CREATE_WORKFLOW_URL, headers=WORKFLOW_HEADERS, json=workflow_payload, verify=False)

    print(f"✅ Created Workflow {batch_num} [{response.status_code}] with {len(batch)} activities")
    print(f"📝 Description: {description_text}")
    print(response.text)
    time.sleep(5)  # avoid same timestamp in name"


🔢 Total Rules find: 303

🗑️ Deleted old file: C:\Users\TapshyuGanvir\OneDrive - Torana Inc\Documents\Scripts\data\81_instance\rules.csv
✅ Saved rules to C:\Users\TapshyuGanvir\OneDrive - Torana Inc\Documents\Scripts\data\81_instance\rules.csv
📥 Loaded 303 rules from CSV
✅ Created Workflow 1 [200] with 5 activities
📝 Description: recon_1760505497489 - recon_1760505492539 - recon_1760505487556 - recon_1760505482619 - recon_1760505477197
{"template":"Sequential","description":"recon_1760505497489 - recon_1760505492539 - recon_1760505487556 - recon_1760505482619 - recon_1760505477197","activities":[{"index":1,"id":"act-e7b5247f-610e-542c-89d7-3c0613b09ddf","name":"recon_1760505497489","type":"ExecuteRule","objectId":"rule-b12439e8-2e59-52c9-8818-91f43b690b02","dependency":{"onSuccess":"continue","onFailure":"continue","onError":"continue"},"properties":{"useExitCode":true,"isActive":true}},{"index":2,"id":"act-25f48c3f-e315-552b-a833-84ce4dc57069","name":"recon_1760505492539","type":"Execu

In [9]:

import requests
import urllib3
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

TOKEN_URL = 'https://192.168.100.76:32222/auth/realms/icedq/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}
token_response = requests.post(TOKEN_URL, data=token_data, verify=False)
access_token = token_response.json().get("access_token")
if not access_token:
    print("❌ Failed to get token.")
    exit()
SEARCH_Workflow_URL = 'https://192.168.100.76:32222/api/v1/workflows/search?pageNo=1&pageSize=2000&sort=updatedTimestamp:desc'
HEADERS = {
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json",
    "org-id": "org-icedq",
    "workspace-id": "wksc-6c9d2689-9b9b-505c-9f30-6c3c68ee5818",
    "Authorization": f"Bearer {access_token}"
}
Workflow_search_PAYLOAD_TEMPLATE = {
    "filter": [
        {
            "attribute": "workflowType",
            "operator": "In",
            "datatype": "string",
            "value": "Custom"
        }
    ]
}
search_workflow_response = requests.post(SEARCH_Workflow_URL, headers=HEADERS, json=Workflow_search_PAYLOAD_TEMPLATE, verify=False)
search_workflow_data = search_workflow_response.json()

post_script_rule_url = "https://192.168.100.76:32222/api/v1/rules"
Rule_HEADERS = {
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json",
    "org-id": "org-icedq",
    "workspace-id": "wksc-6c9d2689-9b9b-505c-9f30-6c3c68ee5818",
    "Authorization": f"Bearer {access_token}",
    "folder-id": "fldr-57afdd2d-e831-5703-98f1-9ac0d047458e"
}
for item in search_workflow_data.get("items", []):
    workflow_id = item["id"]
    workflow_name = item["name"]

    # Base payload
    payload = {
        "template": "script",
        "ruleType": "Script",
        "name": f"Run Custom Workflow_{workflow_name}",
        "description": f"Run Custom Workflow_{workflow_name}",
        "purpose": f"Run Custom Workflow_{workflow_name}",
        "isActive": True,
        "properties": {
            "engineType": "HighThroughput",
            "criticality": "Warning",
            "parameterId": "",
            "parameterName": "",
            "languageType": "groovy"
        },
        "objects": [
            {
                "configuration": {
                    "script": f"def objectId = '{workflow_id}'\r\ndef result = workflow.execute(objectId)\r\ninstanceLog.info('rule executed successfully') \r\nreturn result.ExitCode"
                },
                "id": "",
                "type": "script",
                "name": "script Task"
            },
            {
                "configuration": {
                    "checks": [
                        {
                            "id": "",
                            "name": "exitcode",
                            "expression": {
                                "value": "exitcode < 100"
                            },
                            "isOverride": True,
                            "index": 1
                        }
                    ]
                },
                "id": "",
                "type": "summary",
                "name": "summary"
            }
        ],
        "customFields": []
    }

    # --- Step 3: POST each payload ---
    response = requests.post(post_script_rule_url, headers=Rule_HEADERS, data=json.dumps(payload),verify=False)

    print(f"Posted rule for workflow {workflow_name}: {response.status_code}")
    if response.status_code != 200:
        print("Error response:", response.text)


Posted rule for workflow Workflow_1760505969_61: 200
Posted rule for workflow Workflow_1760505963_60: 200
Posted rule for workflow Workflow_1760505958_59: 200
Posted rule for workflow Workflow_1760505953_58: 200
Posted rule for workflow Workflow_1760505948_57: 200
Posted rule for workflow Workflow_1760505942_56: 200
Posted rule for workflow Workflow_1760505937_55: 200
Posted rule for workflow Workflow_1760505932_54: 200
Posted rule for workflow Workflow_1760505926_53: 200
Posted rule for workflow Workflow_1760505921_52: 200
Posted rule for workflow Workflow_1760505916_51: 200
Posted rule for workflow Workflow_1760505911_50: 200
Posted rule for workflow Workflow_1760505905_49: 200
Posted rule for workflow Workflow_1760505900_48: 200
Posted rule for workflow Workflow_1760505895_47: 200
Posted rule for workflow Workflow_1760505890_46: 200
Posted rule for workflow Workflow_1760505884_45: 200
Posted rule for workflow Workflow_1760505879_44: 200
Posted rule for workflow Workflow_1760505874_4